## 1. Solving differential equations: Euler's method

Groundwater flow and many other geological and physical processes such as heat flow, the propagation of seismic waves, etc. can be described by partial differential equations. Unfortunately partial differential equations are often difficult to solve directly. Analytical solutions of some partial differential equations do exist, but these often require strong simplifications. We will use a numerical method to solve the groundwater flow equation, which is based on a method to approximate derivatives that was first proposed by the Swiss mathematician Leonhard Euler in the 18th century.

The definition of a derivative of a variable $y$ over $x$ is:

$$y'(x) = \frac{\partial y}{\partial x} = \lim_{\Delta x \to 0} \frac{y(x+\Delta x) - y(x)}{\Delta x}$$

For very small values of $\Delta x$ the following is true:

$$\frac{\partial y}{\partial x} \approx \frac{y(x+\Delta x) - y(x)}{\Delta x}$$

Rearranging gives:

$$y(x+\Delta x) \approx y(x) + \frac{\partial y}{\partial x} \Delta x$$

Although for now this seems like a nice, but not terribly useful rearranging of equations, we will show you how to use this method to numerically solve partial differential equations. Euler's method forms the basis of finite-difference methods, which have enabled scientists to solve partial differential equations for which no analytical solutions exist. These methods have been used to predict all kinds of physical processes.

## 2. Numerical solution of the groundwater flow equation

Groundwater flow is described by the following partial differential equation:

$$S_s \frac{\partial h}{\partial t} = - \frac{\partial q}{\partial x} + W$$

where $S_s$ is specific storage ($m^{-1}$), $h$ is the hydraulic head ($m$), $t$ is time ($sec$), $q$ is flux ($m \, s^{-1}$), x is distance ($m$) and $W$ is a term that represents external sources or sinks of water, such as groundwater recharge or pumping ($s^{-1}$).

For the moment we will only look at steady-state groundwater flow, which means that the hydraulic head does not change over time, and therefore $\partial h / \partial t = 0$. And to simplify things a bit more we will assume that all flow is horizontal and we will calculate a vertically integrated flux over the entire depth of the geologic unit for each position $x$.

The vertically integrated version of Darcy's law is:

$$Q = - K b \frac{\partial h}{\partial x}$$

Note that the flux (Q) represents the flux over the entire depth of the aquifer. The units are m² s⁻¹ instead of m s⁻¹.

By combining with Darcy's law we end up with the steady-state groundwater flow equation:

$$K b \frac{\partial ^2 h}{\partial x^2} + W b = 0$$

Where $K$ is hydraulic conductivity ($m s^{-1}$).

## 3. Finite Difference Approximation

The first order derivative of $h$ follows from Euler's method:

$$\frac{\partial h}{\partial x} = \frac{h(x+\Delta x) - h(x)}{\Delta x}$$

The 2nd order derivative can be found by repeating Euler's method:

$$\frac{\partial^2 h}{\partial x^2} = \frac{h(x+\Delta x) + h(x - \Delta x) - 2 h(x)}{\Delta x^2}$$

Inserting our new formulation of the 2nd order derivative into the steady-state flow equation results in:

$$\frac{h(x+\Delta x) + h(x - \Delta x) - 2 h(x)}{\Delta x^2} = - \frac{Wb}{Kb}$$

Rearranging this equation shows how we can find hydraulic head at any point in space ($h(x)$) as a function of the hydraulic head at two adjacent points in space $h(x-\Delta x)$ and $h(x+\Delta x)$:

$$h(x) = \frac{1}{2} \left( \frac{Wb \Delta x^2}{K b} + h(x+\Delta x) + h(x - \Delta x) \right)$$

### Mapping to code

This equation translates directly to Python code. For interior nodes (not at boundaries), we calculate:

```python
h_new[i] = 0.5 * (C[i] + h_old[i+1] + h_old[i-1])
```

where `C = W * dx**2 / K`.

Using array slicing, we can vectorize this for all interior nodes at once:

```python
h_new[1:-1] = 0.5 * (C[1:-1] + h_old[2:] + h_old[:-2])
```

Here:
- `h_new[1:-1]` = all nodes except first and last
- `h_old[2:]` = nodes shifted right (i+1)
- `h_old[:-2]` = nodes shifted left (i-1)

### Check your understanding
1. What does the second derivative $\partial^2 h/\partial x^2$ represent physically in this context?
   - **Hint:** Think about curvature and flow convergence/divergence
   
2. Why do we approximate derivatives with finite differences?
   - **Hint:** Consider when analytical solutions are impossible
   
3. What is the meaning of $W$ and why does it have units of $s^{-1}$?
   - **Hint:** Think about water volume per unit time per unit volume

## 4. Conceptual model: Groundwater flow to a stream

Now let's apply this solution to a groundwater flow problem. Below is a conceptual model of groundwater flow in a permeable rock unit that is bounded by a stream on the left, and is fed by groundwater recharge (shown by $R$). The hydraulic head is a function of the amount of recharge that enters the system, the frictional resistance that the porous rocks exert on the groundwater and the level of the stream.

![Groundwater flow to a stream with uniform recharge](old_material_from_Goettingen/fig/1dim_flow_with_recharge.png)

*Note: If the figure doesn't display, check that the file path is correct relative to this notebook.*

## 5. Boundary conditions (conceptual)

We impose two different boundary conditions:
- **Left boundary (stream):** fixed hydraulic head $h=h_0$.
- **Right boundary (watershed divide):** no-flow, so $\frac{\partial h}{\partial x}=0$.

### Physical meaning

**Fixed head boundary (stream):**
- The stream maintains a constant water level regardless of groundwater flow
- This is realistic for large streams/rivers
- Mathematically: $h(x=0) = h_0$

**No-flow boundary (watershed divide):**
- Water doesn't cross the watershed boundary
- By Darcy's law, zero flux means zero hydraulic gradient
- Mathematically: $\frac{\partial h}{\partial x}\bigg|_{x=L} = 0$

### Implementation in numerical model

This translates to:
- Set the first node equal to $h_0$: `h[0] = h0`
- Set the last node equal to the second-to-last node: `h[-1] = h[-2]` (zero gradient)

## 6. Analytical solution

The conceptual model can be solved analytically with some simplifying assumptions.

The first assumption is that flow is only horizontal ($q_z = 0$). This is somewhat unphysical, but for systems where the overall hydraulic gradient is low, flow is nearly horizontal and the error by neglecting the vertical term in the flow equation is small.

### Derivation

Combining Darcy's law with a mass-balance equation:

$$q = - K b \frac{\partial h}{\partial x}$$

$$q(x) = - R (L-x)$$

Where $R$ is recharge ($m s^{-1}$) and $L$ is the length of the aquifer ($m$).

The mass balance simply states that the steady-state groundwater flux at any point $x$ equals the cumulative recharge from that point to the divide.

Combining these equations:

$$- K b \frac{\partial h}{\partial x} = - R (L-x)$$

Rearranging and integrating:

$$\int K b \, dh = \int R (L-x) \, dx$$

$$K b h = R \left(Lx - \frac{x^2}{2}\right) + C$$

Applying the boundary condition $h = h_0$ at $x = 0$ gives $C = K b h_0$.

### Final solution

After integration and applying the boundary condition $h = h_0$ at $x = 0$, we get:

$$h = h_{0} + \frac{R}{Kb} \left(Lx - \frac{1}{2} x^{2}\right)$$

This parabolic profile results from the linear decrease in flux with distance from the divide.

## Summary of key equations

| Equation | Expression | Purpose |
|----------|-----------|---------|
| **Finite difference** | $h(x) = \frac{1}{2} \left( \frac{Wb \Delta x^2}{K b} + h(x+\Delta x) + h(x - \Delta x) \right)$ | Numerical solution |
| **Analytical solution** | $h = h_{0} + \frac{R}{Kb} \left(Lx - \frac{1}{2} x^{2}\right)$ | Exact solution for validation |
| **Source term** | $W = \frac{R}{b}$ | Converts recharge to solver units |
| **Darcy's law (2D)** | $Q = - K b \frac{\partial h}{\partial x}$ | Flux calculation |

## What's next?

Return to the main exercise notebook to implement these equations in Python and explore groundwater flow behavior through numerical experiments.